In [1]:
from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
import time
from pathlib import Path
from datetime import datetime
import SimpleITK as sitk
from monai.utils import set_determinism
import math
import torch
from torch.utils.data import random_split, DataLoader
import monai
import gdown
import pandas as pd
import torchio as tio
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset,Dataset,PersistentDataset, list_data_collate, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract

sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()

from datetime import datetime
import os
import tempfile
from glob import glob

from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

import torch.nn as nn
import torch.nn.functional as F


sns.set()
plt.rcParams['figure.figsize'] = 12, 8
monai.utils.set_determinism()
import preprocessing.transformsForMain as transformsForMain
import preprocessing.Nyul_preprocessing as Nyul_preprocessing
import preprocessing.manageMetaData as manageMetaData
import dataManag.utils.dataUtils as dataUtils
import multiprocessing

aaa


In [2]:
experiment = Experiment(
    api_key="yB0irIjdk9t7gbpTlSUPnXBd4",
    project_name="picai",
)

comet_logger = CometLogger(
    api_key="yB0irIjdk9t7gbpTlSUPnXBd4",
    workspace="OPI", # Optional
    project_name="picai", # Optional
    #experiment_name="baseline" # Optional
)


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jakubmitura14/picai/ee8b857b9ac742edadce2334bf5cbd35

CometLogger will be initialized in online mode


In [3]:
df= manageMetaData.load_df_only_full()

11471


In [4]:

### Define Data Handling
class PiCaiDataModule(pl.LightningDataModule):
    def __init__(self,trainSizePercent,batch_size,num_workers,drop_last,df,cache_dir):
        super().__init__()
        self.cache_dir=cache_dir
        self.batch_size = batch_size
        self.df = df
        self.num_workers = num_workers
        self.drop_last = drop_last
        self.train_set = None
        self.val_set = None
        self.test_set = None  
        self.trainSizePercent =trainSizePercent
        self.train_files = None
        self.val_files= None
        self.test_files= None
        self.train_ds = None
        self.val_ds= None
        self.test_ds= None        
        self.subjects= None
        self.dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)

    def splitDataSet(self,patList, trainSizePercent):
        """
        test train validation split
        TODO(balance sets)
        """
        totalLen=len(patList)
        train_test_split( patList  )
        numTrain= math.ceil(trainSizePercent*totalLen)
        numTestAndVal=totalLen-numTrain
        numTest=math.ceil(numTestAndVal*0.5)
        numVal= numTestAndVal-numTest

        # valid_set,test_set = torch.utils.data.random_split(test_and_val_set, [math. ceil(0.5), 0.5])
        print('Train data set:', numTrain)
        print('Test data set:',numTest)
        print('Valid data set:', numVal)
        return torch.utils.data.random_split(patList, [numTrain,numVal,numTest])

    def get_preprocessing_transform(self):
        preprocess = tio.Compose([
            tio.RescaleIntensity((-1, 1)),
            tio.CropOrPad(self.get_max_shape(self.subjects + self.test_subjects)),
            tio.EnsureShapeMultiple(8),  # for the U-Net
            tio.OneHot(),
        ])
        return preprocess
    
    def get_augmentation_transform(self):
        augment = tio.Compose([
            tio.RandomAffine(),
            tio.RandomGamma(p=0.5),
            tio.RandomNoise(p=0.5),
            tio.RandomMotion(p=0.1),
            tio.RandomBiasField(p=0.25),
        ])
        return augment

    def setup(self, stage=None):
        set_determinism(seed=0)
        self.subjects = list(map(lambda row: manageMetaData.getMonaiSubjectDataFromDataFrame(row[1])   , list(df.iterrows())))
        train_set, valid_set,test_set = self.splitDataSet(self.subjects , self.trainSizePercent)
        self.train_subjects = train_set
        self.val_subjects = valid_set
        self.test_subjects = test_set
        train_transforms=transformsForMain.get_train_transforms()
        val_transforms= transformsForMain.get_val_transforms()
        
        self.train_ds =  PersistentDataset(data=self.train_subjects, transform=train_transforms,cache_dir=self.cache_dir)
        self.val_ds=     PersistentDataset(data=self.val_subjects, transform=val_transforms,cache_dir=self.cache_dir)
        self.test_ds=    PersistentDataset(data=self.test_subjects, transform=val_transforms,cache_dir=self.cache_dir)    

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, drop_last=self.drop_last
                          , shuffle=True,num_workers=self.num_workers,collate_fn=list_data_collate)#,collate_fn=list_data_collate

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=1, drop_last=False,num_workers=self.num_workers,collate_fn=list_data_collate)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size= 1, drop_last=False,num_workers=self.num_workers,collate_fn=list_data_collate)#num_workers=self.num_workers,

data = PiCaiDataModule(
    df= df,
    batch_size=3,#TODO(batc size determined by lightning)
    trainSizePercent=0.7,
    num_workers=os.cpu_count(),
    drop_last=False,#True,
    cache_dir="/home/sliceruser/labels/preprocess/monai_persistent_Dataset"
)
data.prepare_data()
data.setup()

class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class
    
    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer
    
    # def prepare_batch(self, batch):
    #     #print(len(batch))
    #     if(len(batch)==4):
    #         #print(batch)
    #         return batch[0]['t2w'], batch[0]['label']
    #     return batch['t2w'], batch['label']
    
    # def infer_batch(self, batch):
    #     x, y = self.prepare_batch(batch)
    #     y_hat = self.net(x)
    #     return y_hat, y

    def training_step(self, batch, batch_idx):

        #print(f"in training {len(batch)}")
        
        images, labels = batch["t2w"], batch["label"]
        output = self.net(images)
        loss = self.criterion(output, labels)
        
        # y_hat, y = self.infer_batch(batch)
        # loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):        
        print(f"in val {len(batch)}")      
        print(f"in val image {(batch['t2w'].size() )}") 
        print(f"in val label {(batch['label'].size() )}") 
        
        images, labels = batch["t2w"], batch["label"]
        output = self.net(images)
        loss = self.criterion(output, labels)
        
        # x, y = batch['t2w'], batch['label']
        # y_hat = self.net(x)
        # loss = self.criterion(y_hat, y)
        # self.log('val_loss', loss)
        
        # images, labels = batch["t2w"], batch["label"]
        # output = self.net(images)
        # loss = self.criterion(output, labels)       
        
        
        
        return loss


unet = monai.networks.nets.UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    #channels=(16, 16, 16, 16, 16),
    #strides=(1, 1, 1, 1),
    strides=(2, 2, 2, 2),

    num_res_units=1,
)

model = Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True),
    learning_rate=1e-2,
    optimizer_class=torch.optim.AdamW,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
)
trainer = pl.Trainer(
    #accelerato="cpu", #TODO(remove)
    max_epochs=3,
    #gpus=1,
    precision=16, #TODO(unhash)
    callbacks=[early_stopping],#TODO(unhash)
    logger=comet_logger,
    accelerator='auto',
    devices='auto',
    default_root_dir= "/home/sliceruser/lightning_logs",
)
trainer.logger._default_hp_metric = False


start = datetime.now()
print('Training started at', start)
trainer.fit(model=model, datamodule=data)
print('Training duration:', datetime.now() - start)
experiment.end()

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Train data set: 154
Test data set: 33
Valid data set: 32
Training started at 2022-06-16 15:08:56.800517
Train data set: 154
Test data set: 33
Valid data set: 32


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | net       | UNet       | 2.4 M 
1 | criterion | DiceCELoss | 0     
-----------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
4.896     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

in val 2
in val image torch.Size([1, 1, 1024, 1024, 45])
in val label torch.Size([1, 1, 1024, 1024, 45])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 23 but got size 24 for tensor number 1 in the list.

In [ ]:
experiment.end()